# V19: Alias diagnostics (harmonics + phase-shift events)

Compares the base period against common harmonics (P/2, 2P, …) and searches for high-significance events at phase shifts that can indicate aliasing or structured systematics.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V19')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': list(r.flags),
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "base_depth_ppm_P": 255.9925026659471,
    "base_score_P": 37.6479248900702,
    "best_other_depth_ppm": 216.87980325213462,
    "best_other_harmonic": "2P",
    "best_other_over_base_score_ratio": 0.7162470784918131,
    "best_other_score": 26.965216213791997,
    "max_phase_shift_event_sigma": 16.09151334844518,
    "n_phase_shift_events": 5,
    "secondary_significance_sigma": 0.0
  },
  "status": "ok"
}
```

</details>


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_alias_diagnostics

step_id = '21_v19_alias_diagnostics'
fname = 'V19_alias_diagnostics.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(8, 5))
    plot_alias_diagnostics(r, ax=ax)
    ax.set_title('V19: Alias diagnostics')
    fig.tight_layout()
    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/21_v19_alias_diagnostics/V19_alias_diagnostics.png" width="820" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/21_v19_alias_diagnostics/V19_alias_diagnostics.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/21_v19_alias_diagnostics/V19_alias_diagnostics.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** the strongest competing harmonic is `2P` with score ratio ≈ 0.72; phase-shift event count is 5 with max event ≈ 16.1σ.
- **Why this is useful:** strong performance at harmonics and/or strong phase-shift events can indicate aliasing, repeating artifacts, or non-transit structure.
- **Interpretation:** treat as caution; combine with per-sector diagnostics and robustness sweeps before trusting the inferred period.
- **Next step:** decide on a gated sector set and a consistent detrending choice, then re-check V11–V19 under that configuration.

</details>
